In [ ]:
!pip install pandas==1.3.4

In [ ]:
import pandas as pd

In [ ]:
import pandas as pd

# 시각화 툴
import matplotlib.pyplot as plt
%matplotlib inline

# 한글이 출력되도록 설정합니다.
plt.rc('font', family='Malgun Gothic')

# 선명한 그래픽
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

In [ ]:
# 경기도 진료내역정보 파일 불러오기
filepath = './data'
doctor_data_1 = pd.read_csv(filepath + "/HP_T20_2020_1.CSV", encoding='cp949')  # ①
doctor_data_2 = pd.read_csv(filepath + "/HP_T20_2020_2.CSV", encoding='cp949')
doctor_data_3 = pd.read_csv(filepath + "/HP_T20_2020_3.CSV", encoding='cp949')

doctor_data_1.head(2)

In [ ]:
doctor_data = pd.concat([doctor_data_1, doctor_data_2, doctor_data_3]) #①
target_cols = ['성별코드','연령대코드','진료과목코드','주상병코드','부상병코드']
doctor_data_short = doctor_data[target_cols] #②

print("2020년 진료행위 총 :",doctor_data_short.shape[0], "회") #③
doctor_data_short.head(2)

In [6]:
# 진료과목코드정보 파일 불러오기
# 앞에서 드라이브 경로를 설정(본인의 환경에 맞게 수정)했으므로 data 폴더 경로는 추가하지 않습니다.

treat_code = pd.read_excel(filepath + "/진료과목코드정보.xlsx")

In [7]:
treat_code.dropna(inplace=True)

In [ ]:
treat_code.head(2)

In [9]:
treat_code = treat_code[['코드','명칭']]

In [10]:
treat_code.rename(columns={'코드':'진료과목코드_str', "명칭":"진료과목"}, inplace=True)

In [ ]:
treat_code.head(3)

In [ ]:
code = '1'
print("zfill()에 2로 지정하면 ",code.zfill(2))
print("zfill()에 3로 지정하면 ",code.zfill(3))

In [13]:
treat_code['진료과목코드_str'] = treat_code['진료과목코드_str'].apply(lambda x:str(x).zfill(2))

In [ ]:
treat_code.head(3)

In [ ]:
doctor_data_short['진료과목코드_str']=doctor_data_short['진료과목코드'].apply(lambda x:str(x).zfill(2))

In [ ]:
doctor_data_short.reset_index(drop=True, inplace=True) #①
doctor_data_short.head(1)

In [17]:
doctor_data_short = pd.merge(doctor_data_short, treat_code, how='left', on='진료과목코드_str')

In [ ]:
doctor_data_short.head(2)

In [ ]:
print("2020년 진료과목별 진료횟수 Top20")
doctor_data_short['진료과목'].value_counts()[:20]

In [ ]:
x_hospitals = list(doctor_data_short['진료과목'].value_counts().index) #①
y_hos_values = doctor_data_short['진료과목'].value_counts().values #②

plt.figure(figsize=(6, 10))
plt.barh(x_hospitals, y_hos_values)

In [ ]:
# 건강보험심사평가원 파일 불러오기
# 앞에서 드라이브 경로를 설정(본인의 환경에 맞게 수정)했으므로 data 폴더 경로는 추가하지 않습니다.

disease_code = pd.read_csv (filepath + "/건강보험심사평가원_상병마스터.csv", encoding = 'cp949')
disease_code.head(2)

In [22]:
disease_code = disease_code.iloc[:,:2] #①
disease_code.rename(columns={"상병기호":"주상병코드"}, inplace=True)

In [ ]:
disease_code.head(2)

In [ ]:
doctor_data_short.head(1)

In [25]:
doctor_data_short = pd.merge(doctor_data_short, disease_code, on='주상병코드', how='left')

In [ ]:
doctor_data_short.head(2)

In [27]:
def findSex(num):
    """성별코드를 해석하는 함수"""
    if num==1:
        return "남"
    else:
        return "여"

def ageCode(num):
    """연령대 분류하는 함수"""
    if (num == 1) or (num == 2):
        return "9세미만"
    elif (num == 3) or (num == 4):
        return "10대"
    elif (num == 5) or (num == 6):
        return "20대"
    elif (num == 7) or (num == 8):
        return "30대"
    elif (num == 9) or (num == 10):
        return "40대"
    elif (num == 11) or (num == 12):
        return "50대"
    elif (num == 13) or (num == 14):
        return "60대"
    elif (num == 15) or (num == 16):
        return "70대"
    elif num == 17:
        return "80대"
    else:
        return "80대이상"


In [ ]:
doctor_data_short['성별']=doctor_data_short['성별코드'].apply(lambda x:findSex(num=x)) #①
doctor_data_short['연령대']=doctor_data_short['연령대코드'].apply(lambda x:ageCode(num=x))
doctor_data_short.head(3)

In [29]:
# 2020년 진료정보 파일 저장하기
# 앞에서 드라이브 경로를 설정(본인의 환경에 맞게 수정)했으므로 data 폴더 경로는 추가하지 않습니다.

doctor_data_short.to_csv(filepath + "/2020년_진료정보.csv", index=False)